## Solving set of polynomial equations using Homotopy Continuation

In the following example I have shown how this can be done,  ideally we would want the number of equations to be the same as the number of variables. When this is true n = length of list of input functions. To be on the safer side though, we can enter n manually or depending the the number of point masses we solve for (then n becomes 2n).

In [58]:
using HomotopyContinuation
using LinearAlgebra
using DynamicPolynomials

In [56]:

n = 4 # number of unknown variables

# Define parameters
δ = 0.1    # damping coefficient
ω = 1.5    # frequency
α = 1.0    # linear stiffness
β = 0.04   # nonlinearity
γ = 1.0    # driving force amplitude

# declare the variables 
for i in 1:n÷2
    @polyvar u[i] v[i]
end
@polyvar f[1:n]

# this input funcs will take Hew's function as an input containing all the polynomial equations
input_funcs = [(ω^2 - α)*u1 - δ*ω*v1 + γ - (3/4)*β*(u1^3 + u1*v1^2), ω*δ*u1 + (ω^2 - α)*v1 - (3/4)*β*(v1^3 + u1^2*v1)]

# define the variable f1 to have eqn 1 and f2 to have eqn 2
f = input_funcs

# Define the system as a polynomial system
system = HomotopyContinuation.System(f)

# solve the system
result = HomotopyContinuation.solve(system)

# # display non singular solutions
# for sol in result
#     if HomotopyContinuation.is_nonsingular(sol)
#         println(sol)
#     end
# end

# display real solutions
for sol in result
    if HomotopyContinuation.is_real(sol)
        println(sol)
    end
end



PathResult:
 • return_code → :success
 • solution → ComplexF64[1.3007933510765368 - 1.313717310304516e-44im, 6.402380053229623 + 1.0838167810012257e-45im]
 • accuracy → 7.1016e-15
 • residual → 2.2204e-16
 • condition_jacobian → 148.46
 • steps → 46 / 0
 • extended_precision → false
 • path_number → 2

PathResult:
 • return_code → :success
 • solution → ComplexF64[-1.9999999999999905 - 2.1064858766735392e-35im, 6.0000000000000036 - 8.275480229788904e-36im]
 • accuracy → 1.1054e-20
 • residual → 8.6516e-37
 • condition_jacobian → 93.25
 • steps → 37 / 0
 • extended_precision → false
 • path_number → 7

PathResult:
 • return_code → :success
 • solution → ComplexF64[-0.800793351076541 + 0.0im, 0.09761994677037715 + 5.1510724620250545e-50im]
 • accuracy → 1.0343e-16
 • residual → 9.7578e-18
 • condition_jacobian → 1.2112
 • steps → 17 / 0
 • extended_precision → false
 • path_number → 8



This was run with 1 mass, hence one set of parameter values, in the case this is not true we can write a function to take a list containing these values and then later use in the polynomial equations. 

In [76]:
n = 4 # number of unknown variables

# declare the variables 
@polyvar u[1:n÷2] v[1:n÷2]
@polyvar f[1:n]

# declare the parameters
@polyvar α[1:n÷2]
@polyvar β[1:n÷2]
@polyvar γ[1:n÷2]
@polyvar δ[1:n÷2]


# this input funcs will take Hew's function as an input containing all the polynomial equations
input_alpha = [1.0, 1.1]
input_beta = [0.04, 0.03]
input_gamma = [1.0, 1.1]
input_delta = [0.1, 0.2]
input_omega = [1.5]

α = input_alpha
β = input_beta
γ = input_gamma
δ = input_delta
ω = input_omega

input_function = [
    (ω[1]^2 - α[1])*u[1] - δ[1]*ω[1]*v[1] + γ[1] - (3/4)*β[1]*(u[1]^3 + u[1]*v[1]^2),
    (ω[1]^2 - α[2])*u[2] - δ[2]*ω[1]*v[2] + γ[2] - (3/4)*β[2]*(u[2]^3 + u[2]*v[2]^2),
    ω[1]*δ[1]*u[1] + (ω[1]^2 - α[1])*v[1] - (3/4)*β[1]*(v[1]^3 + u[1]^2*v[1]),
    ω[1]*δ[2]*u[2] + (ω[1]^2 - α[2])*v[2] - (3/4)*β[2]*(v[2]^3 + u[2]^2*v[2])
]

f = input_function

# Define the system as a polynomial system
system = HomotopyContinuation.System(f)

# solve the system
result = HomotopyContinuation.solve(system)

# display real solutions
for sol in result
    if HomotopyContinuation.is_real(sol)
        println(sol)
    end
end









Tracking 81 paths...   2%|█                             |  ETA: 0:02:14



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  81
  # non-singular solutions (real):  9 (3)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         9 (3)
PathResult:
 • return_code → :success
 • solution → ComplexF64[-1.999999999999991 - 5.583598166405866e-38im, -0.9092972094148429 + 0.0im, 6.0000000000000036 - 3.4530146555404696e-38im, 0.2413880779738752 - 9.683056929885334e-70im]
 • accuracy → 2.6378e-15
 • residual → 1.3878e-17
 • condition_jacobian → 87.178
 • steps → 79 / 0
 • extended_precision → false
 • path_number → 55

PathResult:
 • return_code → :success
 • solution → ComplexF64[-0.8007933510765409 - 2.2958874039497803e-41im, -0.9092972094148429 - 2.2420775429197073e-44im, 0.09761994677037714 + 1.4349296274686127e-42im, 0.2413880779738752 + 0.0im]
 • accuracy → 1.4048e-16
 • residual → 1.3878e-17
 • condition_jacobian 